##Variables

In [0]:
# Widgets
dbutils.widgets.text("storage_account_name", "SUSTITUIR", "Cuenta de almacenamiento")
dbutils.widgets.text("storage_account_key", "SUSTITUIR", "Clave de la cuenta")
dbutils.widgets.text("container_name", "datalake", "Nombre del contenedor")
dbutils.widgets.text("file_path", "bronze/transactions.csv", "Ruta del archivo en bronze")

In [0]:
# Leer la clave secreta desde un Azure Key Vault Scope
# storage_account_key = dbutils.secrets.get(scope="my_scope", key="storage_account_key")

In [0]:
# Obtener los valores de los widgets
storage_account_name = dbutils.widgets.get("storage_account_name")
storage_account_key = dbutils.widgets.get("storage_account_key")
container_name = dbutils.widgets.get("container_name")
file_path = dbutils.widgets.get("file_path")

##Leer los datos en RAW (**Bronze**)

In [0]:
# Establecer la clave de la cuenta de almacenamiento en la configuración de Spark
spark.conf.set(f"fs.azure.account.key.{storage_account_name}.dfs.core.windows.net", storage_account_key)

# Leer el archivo CSV desde el Data Lake con inferencia de esquema y separador ;
df = spark.read.format("csv").option("header", "true").option("inferSchema", "true").option("sep", ";").load(f"abfss://{container_name}@{storage_account_name}.dfs.core.windows.net/{file_path}")

# Mostrar el esquema del DataFrame
df.printSchema()

# Mostrar el DataFrame
display(df)

# Escribir los datos en formato Delta (Silver)

In [0]:
from pyspark.sql.functions import current_date

# Agregar la columna load_date
df_with_date = df.withColumn("load_date", current_date())

# Escribir el DataFrame en formato Delta en el Data Lake, particionado por load_date
df_with_date.write.format("delta").mode("overwrite").option("overwriteSchema", "true").partitionBy("load_date").save(f"abfss://{container_name}@{storage_account_name}.dfs.core.windows.net/silver/h_transactions")

# Generar tabla h_transactions en Unity

In [0]:
# Crear el esquema si no existe
spark.sql("CREATE SCHEMA IF NOT EXISTS silver")

# Crear la tabla externa en Unity Catalog
spark.sql(f"""
CREATE TABLE IF NOT EXISTS silver.h_transactions
USING DELTA
LOCATION 'abfss://{container_name}@{storage_account_name}.dfs.core.windows.net/silver/h_transactions'""")